In [1]:
import pandas as pd
import numpy as np
from itertools import combinations
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor
import warnings
import re
import ast

warnings.filterwarnings('ignore')


In [2]:
lift_df = pd.read_csv("lifts.csv", index_col=0)
data_for_lift = pd.read_csv("data_for_lift.csv", index_col=0)

In [50]:
lift_df

,Feature Combination,Feature,Lift Value
Cancer Type,,,
Breast Carcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Male', 'Unknown', '17', 'TP53', 'C>T')",0.00
Breast Carcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Male', 'Unknown', '17', 'TP53', 'G>A')",0.00
Breast Carcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Male', 'Unknown', '17', 'TP53', 'C>A')",0.00
Breast Carcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Male', 'Unknown', '5', 'TERT', 'G>A')",0.00
Breast Carcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Male', 'Unknown', '12', 'KRAS', 'C>T')",0.00
...,...,...,...
Soft Tissue Sarcoma,"('TMB (nonsynonymous)', 'Position', 'Protein_p...","(0.166666667, '25398284.0-25398284.0', 12.0, '...",0.00
Soft Tissue Sarcoma,"('TMB (nonsynonymous)', 'Position', 'Protein_p...","(0.233333333, '25398284.0-25398284.0', 12.0, '...",0.63
Soft Tissue Sarcoma,"('TMB (nonsynonymous)', 'Position', 'Protein_p...","(0.1, '55259515.0-55259515.0', 858.0, 'cTg/cGg...",0.00


In [51]:
# try_df = data_for_lift[data_for_lift["Cancer Type"] == "Colorectal Carcinoma"]
# try_df = data_for_lift[data_for_lift['Smoke Status'] == "Unknown"]
# try_df = try_df[try_df['SNP_event'] == "C>G"]
# try_df = try_df[try_df['Exon_Number'] == "02/05"]
# try_df = try_df[try_df['Protein_position'] == 12.0]
# try_df = try_df[try_df['Codons'] == "Ggt/Cgt"]
#
# set(try_df.index)
# try_df

In [52]:
lift_df.sort_values(by="Lift Value", ascending=False).head(1000)

,Feature Combination,Feature,Lift Value
Cancer Type,,,
Pancreatic Cancer,"('Smoke Status', 'Chromosome', 'SNP_event', 'P...","('Unknown', '12', 'C>G', 12.0, 'Ggt/Cgt')",27.03
Pancreatic Cancer,"('Smoke Status', 'Hugo_Symbol', 'Consequence',...","('Unknown', 'KRAS', 'missense_variant', 12.0, ...",27.03
Pancreatic Cancer,"('Smoke Status', 'Hugo_Symbol', 'SNP_event', '...","('Unknown', 'KRAS', 'C>G', '25398285.0-2539828...",27.03
Pancreatic Cancer,"('Smoke Status', 'Hugo_Symbol', 'SNP_event', '...","('Unknown', 'KRAS', 'C>G', 'Ggt/Cgt', 'Substit...",27.03
Pancreatic Cancer,"('Smoke Status', 'Chromosome', 'SNP_event', 'P...","('Unknown', '12', 'C>G', '25398285.0-25398285....",27.03
...,...,...,...
Pancreatic Cancer,"('Smoke Status', 'Consequence', 'Exon_Number',...","('Unknown', 'missense_variant', '02/05', '51-6...",16.64
Pancreatic Cancer,"('Chromosome', 'Hugo_Symbol', 'Exon_Number', '...","('12', 'KRAS', '02/05', 0.133333333, 12.0)",16.64
Pancreatic Cancer,"('Smoke Status', 'SNP_event', 'Diagnosis Age',...","('Unknown', 'C>A', '51-60', '25398284.0-253982...",16.64


In [53]:
df = lift_df.copy()

In [54]:
df.reset_index(inplace=True)

In [55]:
df.sort_values(by="Lift Value", ascending=False)

,Cancer Type,Feature Combination,Feature,Lift Value
411834,Pancreatic Cancer,"('Smoke Status', 'Chromosome', 'SNP_event', 'P...","('Unknown', '12', 'C>G', 12.0, 'Ggt/Cgt')",27.03
415593,Pancreatic Cancer,"('Smoke Status', 'Hugo_Symbol', 'Consequence',...","('Unknown', 'KRAS', 'missense_variant', 12.0, ...",27.03
414655,Pancreatic Cancer,"('Smoke Status', 'Hugo_Symbol', 'SNP_event', '...","('Unknown', 'KRAS', 'C>G', '25398285.0-2539828...",27.03
414833,Pancreatic Cancer,"('Smoke Status', 'Hugo_Symbol', 'SNP_event', '...","('Unknown', 'KRAS', 'C>G', 'Ggt/Cgt', 'Substit...",27.03
411730,Pancreatic Cancer,"('Smoke Status', 'Chromosome', 'SNP_event', 'P...","('Unknown', '12', 'C>G', '25398285.0-25398285....",27.03
...,...,...,...,...
210620,Gallbladder Carcinoma,"('Hugo_Symbol', 'SNP_event', 'Consequence', 'E...","('TP53', 'C>T', 'missense_variant', '07/11', '...",0.00
210621,Gallbladder Carcinoma,"('Hugo_Symbol', 'SNP_event', 'Consequence', 'E...","('KRAS', 'C>T', 'missense_variant', '02/05', '...",0.00
210624,Gallbladder Carcinoma,"('Hugo_Symbol', 'SNP_event', 'Consequence', 'E...","('EGFR', 'T>G', 'missense_variant', '21/28', 0...",0.00
210626,Gallbladder Carcinoma,"('Hugo_Symbol', 'SNP_event', 'Consequence', 'E...","('KRAS', 'C>T', 'missense_variant', '02/05', 0...",0.00


In [56]:
# Improved function to safely parse tuple-like strings
def safe_eval(val):
    if pd.isna(val) or val == 'None' or val == 'nan':
        return None

    try:
        # Handle the case where it's already a proper string representation of a tuple
        return ast.literal_eval(val)
    except (ValueError, SyntaxError):
        try:
            # Fix missing quotes around words and try again
            fixed_val = re.sub(r'(\b\w+\b)(?=\s*,|\s*\))', r"'\1'", val)
            fixed_val = re.sub(r'\(\s*,', '(', fixed_val)  # Fix empty first item
            fixed_val = re.sub(r',\s*\)', ')', fixed_val)  # Fix empty last item
            return ast.literal_eval(fixed_val)
        except (ValueError, SyntaxError):
            # If still fails, try to extract content between parentheses
            match = re.search(r'\((.*)\)', val)
            if match:
                items = match.group(1).split(',')
                # Clean and convert items
                processed_items = []
                for item in items:
                    item = item.strip()
                    if item and item != 'None':
                        # Try to convert to appropriate type
                        try:
                            if item.lower() == 'true':
                                processed_items.append(True)
                            elif item.lower() == 'false':
                                processed_items.append(False)
                            elif item.replace('.', '', 1).isdigit():
                                if '.' in item:
                                    processed_items.append(float(item))
                                else:
                                    processed_items.append(int(item))
                            else:
                                processed_items.append(item.strip("'\""))
                        except:
                            processed_items.append(item.strip("'\""))

                return tuple(processed_items) if processed_items else None
            return None

# Apply parsing with tqdm progress bar
tqdm.pandas(desc="Parsing Features")
df["Feature"] = df["Feature"].progress_apply(lambda x: safe_eval(str(x)) if not pd.isna(x) else None)
df["Feature Combination"] = df["Feature Combination"].progress_apply(lambda x: safe_eval(str(x)) if not pd.isna(x) else None)

Parsing Features: 100%|██████████| 516372/516372 [00:03<00:00, 161845.87it/s]


In [57]:
# Find all unique features across the dataset
all_features = set()
for features in df["Feature Combination"].dropna():
    if isinstance(features, tuple):  # Make sure it's actually a tuple
        all_features.update(features)

# Prepare transformed data
expanded_data = []

for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing Rows"):
    new_row = {
        "Cancer Type": row["Cancer Type"],
        "Lift Value": row["Lift Value"]
    }

    # The bug is here: Check if both are tuples AND verify they're the same length
    if isinstance(row["Feature Combination"], tuple) and isinstance(row["Feature"], tuple):
        if len(row["Feature Combination"]) == len(row["Feature"]):
            new_row.update(dict(zip(row["Feature Combination"], row["Feature"])))
        else:
            # Handle case where tuples are different lengths
            # Only use the matching portions
            min_length = min(len(row["Feature Combination"]), len(row["Feature"]))
            new_row.update(dict(zip(row["Feature Combination"][:min_length],
                                    row["Feature"][:min_length])))

    # Assign NaN to missing feature columns
    for feature in all_features:
        if feature not in new_row:
            new_row[feature] = None  # Using None to represent NaN

    # Only append rows that have at least one non-null feature value
    if any(new_row.get(feature) is not None for feature in all_features):
        expanded_data.append(new_row)
    else:
        # Optionally include even empty rows if that's what you want
        expanded_data.append(new_row)

# Convert to DataFrame
df_transformed = pd.DataFrame(expanded_data)

Processing Rows: 100%|██████████| 516372/516372 [00:07<00:00, 70680.26it/s]


In [58]:
# d = df_transformed[df_transformed["Protein_position"] == 249]
# d[d["Cancer Type"] == "Intrahepatic Cholangiocarcinoma"]
df_transformed = df_transformed.drop_duplicates()
df_transformed = df_transformed[df_transformed["Lift Value"] > 6]
df_transformed

,Cancer Type,Lift Value,Sex,Smoke Status,Chromosome,Hugo_Symbol,SNP_event,Diagnosis Age,VAR_TYPE_SX,Codons,Protein_position,Position,Consequence,Exon_Number,TMB (nonsynonymous)
13,Breast Carcinoma,6.49,Female,Unknown,17,TP53,C>A,None,None,None,NaN,None,None,None,NaN
17,Breast Carcinoma,11.35,Female,Unknown,3,PIK3CA,G>A,None,None,None,NaN,None,None,None,NaN
28,Breast Carcinoma,22.01,Female,Unknown,3,PIK3CA,A>G,None,None,None,NaN,None,None,None,NaN
105,Breast Carcinoma,8.77,Female,Unknown,17,TP53,A>G,None,None,None,NaN,None,None,None,NaN
126,Breast Carcinoma,15.30,Female,Unknown,3,PIK3CA,None,None,None,None,NaN,None,missense_variant,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515961,Soft Tissue Sarcoma,9.40,None,None,None,None,None,None,Substitution/Indel,None,41.0,41266124.0-41266124.0,missense_variant,03/15,NaN
515993,Soft Tissue Sarcoma,9.40,None,None,None,None,None,None,Substitution/Indel,Acc/Gcc,NaN,41266124.0-41266124.0,missense_variant,03/15,NaN
516025,Soft Tissue Sarcoma,9.40,None,None,None,None,None,None,Substitution/Indel,Acc/Gcc,41.0,None,missense_variant,03/15,NaN
516183,Soft Tissue Sarcoma,9.40,None,None,None,None,None,None,Substitution/Indel,Acc/Gcc,41.0,41266124.0-41266124.0,missense_variant,None,NaN


In [16]:
# def find_matching_patients(df_transformed, data_for_lift):
#
#     # Ensure types match across dataframes (optional, depends on your data)
#     df_transformed = df_transformed.copy()
#     data_for_lift = data_for_lift.copy()
#
#     # Pre-index data_for_lift by Cancer Type to reduce row scans
#     cancer_type_groups = data_for_lift.groupby('Cancer Type')
#
#     # Result list to store matched PATIENT_IDs
#     matched_ids = []
#
#     # Iterate through df_transformed
#     for idx, row in df_transformed.iterrows():
#         cancer_type = row['Cancer Type']
#         if cancer_type not in cancer_type_groups.groups:
#             matched_ids.append("")
#             continue
#
#         subset = cancer_type_groups.get_group(cancer_type)
#
#         # Prepare filters: only columns with non-null values, excluding meta-columns
#         filter_columns = row.drop(labels=['Cancer Type', 'Lift Value']).dropna()
#
#         if filter_columns.empty:
#             # If no filters, return all patient IDs for this cancer type
#             matched = subset['PATIENT_ID'].dropna().unique()
#         else:
#             # Start with all True boolean mask
#             mask = pd.Series(True, index=subset.index)
#             for col, val in filter_columns.items():
#                 if col in subset.columns:
#                     mask &= subset[col] == val
#             matched = subset.loc[mask, 'PATIENT_ID'].dropna().unique()
#
#         # Append as comma-separated string
#         matched_ids.append(','.join(map(str, matched)))
#
#     # Add to DataFrame
#     df_transformed['Matched PATIENT_IDs'] = matched_ids
#     return df_transformed
#
#
# df_result = find_matching_patients(df_transformed, data_for_lift)

def find_matching_patients(df_transformed, data_for_lift):
    df_transformed = df_transformed.copy()
    data_for_lift = data_for_lift.copy()
    cancer_type_groups = data_for_lift.groupby('Cancer Type')
    matched_ids = []
    matched_counts = []
    total_matched_feature_counts = []

    for idx, row in df_transformed.iterrows():
        cancer_type = row['Cancer Type']
        filter_columns = row.drop(labels=['Cancer Type', 'Lift Value']).dropna()

        # Find matched patient IDs for this cancer type
        if cancer_type not in cancer_type_groups.groups:
            matched = []
        else:
            subset = cancer_type_groups.get_group(cancer_type)
            if filter_columns.empty:
                matched = subset['PATIENT_ID'].dropna().unique()
            else:
                mask = pd.Series(True, index=subset.index)
                for col, val in filter_columns.items():
                    if col in subset.columns:
                        mask &= subset[col] == val
                matched = subset.loc[mask, 'PATIENT_ID'].dropna().unique()
        matched_ids.append(','.join(map(str, matched)))
        matched_counts.append(len(matched))

        # Find total matched patient IDs with these features across all cancer types
        if filter_columns.empty:
            total_matched = data_for_lift['PATIENT_ID'].dropna().unique()
        else:
            mask = pd.Series(True, index=data_for_lift.index)
            for col, val in filter_columns.items():
                if col in data_for_lift.columns:
                    mask &= data_for_lift[col] == val
            total_matched = data_for_lift.loc[mask, 'PATIENT_ID'].dropna().unique()
        total_matched_feature_counts.append(len(total_matched))

    df_transformed['Matched PATIENT_IDs'] = matched_ids
    df_transformed['Matched_Count'] = matched_counts
    df_transformed['Total_Matched_Feature_Count'] = total_matched_feature_counts
    return df_transformed
# df_result = find_matching_patients(df_transformed, data_for_lift)


In [160]:
# Calculate unique patient counts per cancer type
cancer_type_patient_counts = data_for_lift.groupby('Cancer Type')['PATIENT_ID'].nunique()
# Map to your result DataFrame
df_result['Total_Unique_Patients_Cancer_Type'] = df_result['Cancer Type'].map(cancer_type_patient_counts)
total_patients = data_for_lift['PATIENT_ID'].nunique()
df_result['Total_Patient_IDs'] = total_patients
P_A = df_result['Total_Unique_Patients_Cancer_Type'] / total_patients
P_B = df_result['Total_Matched_Feature_Count'] / total_patients
P_A_B = df_result['Matched_Count'] / total_patients
df_result["lift_try"] = P_A_B / (P_A * P_B)
df_result

,Cancer Type,Lift Value,Sex,Smoke Status,Chromosome,Hugo_Symbol,SNP_event,Diagnosis Age,VAR_TYPE_SX,Codons,...,Position,Consequence,Exon_Number,TMB (nonsynonymous),Matched PATIENT_IDs,Matched_Count,Total_Matched_Feature_Count,Total_Unique_Patients_Cancer_Type,lift_try,Total_Patient_IDs
13,Breast Carcinoma,6.49,Female,Unknown,17,TP53,C>A,None,None,None,...,None,None,None,NaN,"Patient0160,Patient0417,Patient0725,Patient180...",19,175,313,2.905066,8375
17,Breast Carcinoma,11.35,Female,Unknown,3,PIK3CA,G>A,None,None,None,...,None,None,None,NaN,"Patient0021,Patient0129,Patient0386,Patient058...",39,156,313,6.689297,8375
28,Breast Carcinoma,22.01,Female,Unknown,3,PIK3CA,A>G,None,None,None,...,None,None,None,NaN,"Patient0041,Patient0074,Patient0114,Patient016...",56,105,313,14.270501,8375
105,Breast Carcinoma,8.77,Female,Unknown,17,TP53,A>G,None,None,None,...,None,None,None,NaN,"Patient3765,Patient4287,Patient5053,Patient594...",9,51,313,4.721857,8375
126,Breast Carcinoma,15.30,Female,Unknown,3,PIK3CA,None,None,None,None,...,None,missense_variant,None,NaN,"Patient0021,Patient0041,Patient0074,Patient011...",113,327,313,9.246368,8375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515961,Soft Tissue Sarcoma,9.40,None,None,None,None,None,None,Substitution/Indel,None,...,41266124.0-41266124.0,missense_variant,03/15,NaN,"Patient0895,Patient2250,Patient2609,Patient539...",8,51,454,2.893668,8375
515993,Soft Tissue Sarcoma,9.40,None,None,None,None,None,None,Substitution/Indel,Acc/Gcc,...,41266124.0-41266124.0,missense_variant,03/15,NaN,"Patient0895,Patient2250,Patient2609,Patient539...",8,51,454,2.893668,8375
516025,Soft Tissue Sarcoma,9.40,None,None,None,None,None,None,Substitution/Indel,Acc/Gcc,...,None,missense_variant,03/15,NaN,"Patient0895,Patient2250,Patient2609,Patient539...",8,51,454,2.893668,8375
516183,Soft Tissue Sarcoma,9.40,None,None,None,None,None,None,Substitution/Indel,Acc/Gcc,...,41266124.0-41266124.0,missense_variant,None,NaN,"Patient0895,Patient2250,Patient2609,Patient539...",8,51,454,2.893668,8375


In [158]:
t = data_for_lift[data_for_lift["Smoke Status"] == "Unknown"]
t = t[t["Chromosome"] == "17"]
t = t[t["Hugo_Symbol"] == "TP53"]
# t = t[t["VAR_TYPE_SX"] == 'Substitution/Indel']
# t = t[t["Exon_Number"] == "21/21"]
# t = t[t["Codons"] == "cAt/cGt"]
t = t[t["SNP_event"] == "C>A"]
t = t[t["Sex"] == "Female"]
# t = t[t["Consequence"] == "upstream_gene_variant"]
len(list(t["PATIENT_ID"].unique()))

175

In [163]:
df_result = df_result.sort_values(by="lift_try", ascending=False)
unique_ids = df_result['Matched PATIENT_IDs'].drop_duplicates().reset_index(drop=True)
id_to_rank = {pid: i+1 for i, pid in unique_ids.items()}
df_result['patient_rank'] = df_result['Matched PATIENT_IDs'].map(id_to_rank)
df_result

,Cancer Type,Lift Value,Sex,Smoke Status,Chromosome,Hugo_Symbol,SNP_event,Diagnosis Age,VAR_TYPE_SX,Codons,...,Consequence,Exon_Number,TMB (nonsynonymous),Matched PATIENT_IDs,Matched_Count,Total_Matched_Feature_Count,Total_Unique_Patients_Cancer_Type,lift_try,Total_Patient_IDs,patient_rank
431475,Small Cell Lung Cancer,20.05,Male,Smoker,13,RB1,None,None,Truncation,None,...,None,None,NaN,"Patient0238,Patient0492,Patient0632,Patient070...",47,71,215,25.786112,8375,1
431572,Small Cell Lung Cancer,13.66,Male,Smoker,13,RB1,None,None,Substitution/Indel,None,...,None,None,NaN,"Patient0125,Patient0492,Patient0978,Patient118...",23,51,215,17.567259,8375,2
9956,Breast Carcinoma,26.22,Female,Unknown,None,None,None,None,Substitution/Indel,cAt/cGt,...,None,None,NaN,"Patient0041,Patient0074,Patient0114,Patient016...",55,86,313,17.112155,8375,3
1929,Breast Carcinoma,26.22,Female,Unknown,3,None,A>G,None,None,None,...,None,None,NaN,"Patient0041,Patient0074,Patient0114,Patient016...",55,86,313,17.112155,8375,3
4513,Breast Carcinoma,26.22,Female,Unknown,None,PIK3CA,A>G,None,None,None,...,None,None,NaN,"Patient0041,Patient0074,Patient0114,Patient016...",55,86,313,17.112155,8375,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498689,Soft Tissue Sarcoma,6.08,None,Unknown,21,None,None,51-60,Substitution/Indel,None,...,missense_variant,None,NaN,"Patient1054,Patient5393,Patient6929",3,55,454,1.006207,8375,1439
479713,Soft Tissue Sarcoma,6.99,Male,Unknown,None,None,C>G,None,None,None,...,missense_variant,None,0.266667,"Patient4108,Patient4200,Patient5393,Patient8219",4,76,454,0.970902,8375,1440
180569,Gallbladder Carcinoma,6.52,Male,Unknown,None,None,None,51-60,None,None,...,frameshift_variant,None,0.166667,Patient4688,1,56,239,0.625747,8375,1441
488523,Soft Tissue Sarcoma,6.93,Male,None,12,None,None,61-70,Substitution/Indel,None,...,None,None,0.166667,Patient1290,1,61,454,0.302412,8375,1442


In [164]:
# Group by 'patient_rank' and aggregate unique values as comma-separated strings
agg_df = df_result.groupby('patient_rank').agg(
    lambda x: ','.join(sorted(map(str, pd.unique(x.dropna()))))
).reset_index()

agg_df

,patient_rank,Cancer Type,Lift Value,Sex,Smoke Status,Chromosome,Hugo_Symbol,SNP_event,Diagnosis Age,VAR_TYPE_SX,...,Position,Consequence,Exon_Number,TMB (nonsynonymous),Matched PATIENT_IDs,Matched_Count,Total_Matched_Feature_Count,Total_Unique_Patients_Cancer_Type,lift_try,Total_Patient_IDs
0,1,Small Cell Lung Cancer,20.05,Male,Smoker,13,RB1,,,Truncation,...,,,,,"Patient0238,Patient0492,Patient0632,Patient070...",47,71,215,25.78611202096299,8375
1,2,Small Cell Lung Cancer,13.66,Male,Smoker,13,RB1,,,Substitution/Indel,...,,,,,"Patient0125,Patient0492,Patient0978,Patient118...",23,51,215,17.567259461924305,8375
2,3,Breast Carcinoma,"20.42,22.24,23.73,23.95,24.19,24.42,24.67,25.1...",Female,Unknown,3,PIK3CA,A>G,,Substitution/Indel,...,178952085.0-178952085.0,missense_variant,21/21,,"Patient0041,Patient0074,Patient0114,Patient016...",55,"100,110,86,87,88,90,92,93,94,95,96",313,"13.378594249201278,14.716453674121404,15.32963...",8375
3,4,Small Cell Lung Cancer,13.26,Male,Smoker,13,RB1,,61-70,,...,,,,,"Patient0125,Patient1305,Patient2647,Patient323...",22,52,215,16.480322003577818,8375
4,5,Breast Carcinoma,"23.19,23.58,23.77,25.25,25.47",Female,Unknown,3,PIK3CA,,,Substitution/Indel,...,178952085.0-178952085.0,missense_variant,21/21,,"Patient0041,Patient0074,Patient0114,Patient016...",59,"106,109,98,99",313,"14.483248820236245,14.893152088733498,15.94620...",8375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1437,1438,Soft Tissue Sarcoma,6.73,Male,Unknown,,,C>G,,Substitution/Indel,...,,,,0.266666667,"Patient4108,Patient4200,Patient5393,Patient821...",5,90,454,1.0248409202153697,8375
1438,1439,Soft Tissue Sarcoma,6.08,,Unknown,21,,,51-60,Substitution/Indel,...,,missense_variant,,,"Patient1054,Patient5393,Patient6929",3,55,454,1.0062074489387265,8375
1439,1440,Soft Tissue Sarcoma,6.99,Male,Unknown,,,C>G,,,...,,missense_variant,,0.266666667,"Patient4108,Patient4200,Patient5393,Patient8219",4,76,454,0.9709019244145605,8375
1440,1441,Gallbladder Carcinoma,6.52,Male,Unknown,,,,51-60,,...,,frameshift_variant,,0.166666667,Patient4688,1,56,239,0.6257471607890018,8375


In [138]:
agg_df["support"] = 0
for idx, row in agg_df.iterrows():
    agg_df.at[idx, "support"] = len(str(row['Matched PATIENT_IDs']).split(","))

In [139]:
agg_df

,patient_rank,Cancer Type,Lift Value,Sex,Smoke Status,Chromosome,Hugo_Symbol,SNP_event,Diagnosis Age,VAR_TYPE_SX,Codons,Protein_position,Position,Consequence,Exon_Number,TMB (nonsynonymous),Matched PATIENT_IDs,support
0,1,Pancreatic Cancer,"14.95,15.97,16.47,17.01,18.5,19.17,19.71,19.89...",,Unknown,12,KRAS,C>G,,Substitution/Indel,Ggt/Cgt,12.0,25398285.0-25398285.0,missense_variant,02/05,,"Patient0257,Patient0557,Patient0653,Patient068...",66
1,2,Breast Carcinoma,"20.42,22.24,23.73,23.95,24.19,24.42,24.67,25.1...",Female,Unknown,3,PIK3CA,A>G,,Substitution/Indel,cAt/cGt,1047.0,178952085.0-178952085.0,missense_variant,21/21,,"Patient0041,Patient0074,Patient0114,Patient016...",55
2,3,Breast Carcinoma,"23.19,23.58,23.77,25.25,25.47",Female,Unknown,3,PIK3CA,,,Substitution/Indel,,1047.0,178952085.0-178952085.0,missense_variant,21/21,,"Patient0041,Patient0074,Patient0114,Patient016...",59
3,4,Pancreatic Cancer,"10.03,10.94,11.45,18.97,20.58,20.93,22.07,22.9...",Female,Unknown,12,KRAS,C>G,,Substitution/Indel,Ggt/Cgt,12.0,,missense_variant,02/05,,"Patient0653,Patient0689,Patient10144,Patient10...",38
4,5,Breast Carcinoma,"21.53,23.73",Female,Unknown,3,PIK3CA,,,Substitution/Indel,,,,missense_variant,21/21,,"Patient0041,Patient0074,Patient0114,Patient016...",63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1437,1438,Small Cell Lung Cancer,6.01,Male,,12,,C>A,,Truncation,,,,stop_gained,,,"Patient0238,Patient1305,Patient2287,Patient290...",10
1438,1439,Small Cell Lung Cancer,6.01,,Smoker,17,,C>A,61-70,,,,,missense_variant,,,"Patient1305,Patient3231,Patient6201,Patient632...",11
1439,1440,Extrahepatic Cholangiocarcinoma,6.01,,Unknown,18,SMAD4,,,Truncation,,,,frameshift_variant,,,"Patient0447,Patient0964,Patient10038,Patient11...",18
1440,1441,Gastric Cancer,6.01,,Unknown,3,RHOA,,,Substitution/Indel,,,,missense_variant,,,"Patient0472,Patient10130,Patient10142,Patient1...",49


In [150]:
num_patient_cancer = len(list(data_for_lift[data_for_lift["Cancer Type"] == "Liver Hepatocellular Carcinoma"]["PATIENT_ID"].unique()))
total_patients = len(list(data_for_lift["PATIENT_ID"].unique()))
num_patient_cancer / total_patients



0.13397014925373135

In [79]:
import pandas as pd
import numpy as np
from collections import defaultdict
from typing import List, Tuple, Dict, Any

def combine_with_lift_tolerance(df: pd.DataFrame, lift_tolerance: float = 1.0) -> pd.DataFrame:
    """
    Takes the result of combine_cancer_data_rows and performs another round of combining
    for rows that match all conditions but differ by up to lift_tolerance in lift value.
    """

    # Create a copy to avoid modifying original data
    df = df.copy()

    # Identify columns to exclude from matching (index-like columns)
    id_cols = []
    if len(df.columns) > 0:
        first_col = df.columns[0]
        if (df[first_col].dtype in ['int64', 'float64', 'object'] and
            (first_col.lower() in ['id', 'index'] or first_col == df.columns[0])):
            id_cols = [first_col]

    matching_cols = [col for col in df.columns if col not in id_cols + ['Cancer Type', 'Lift Value']]

    # Group by Cancer Type first
    cancer_groups = df.groupby('Cancer Type')

    combined_rows = []
    processed_indices = set()

    for cancer_type, cancer_group in cancer_groups:
        # Within each cancer type, find lift value groups with tolerance
        lift_groups = find_lift_value_groups_with_tolerance(cancer_group, lift_tolerance)

        for lift_group_indices in lift_groups:
            lift_group = cancer_group.loc[lift_group_indices]

            # Skip if any row in this lift group is already processed
            if any(idx in processed_indices for idx in lift_group_indices):
                continue

            # Find combinable row groups within this lift value group
            combinable_groups = find_combinable_groups_tolerance(lift_group, matching_cols)

            for row_group in combinable_groups:
                if any(idx in processed_indices for idx in row_group):
                    continue

                # Combine the rows in this group
                combined_row = combine_row_group_with_lift_avg(lift_group.loc[row_group], id_cols[0] if id_cols else None)
                combined_rows.append(combined_row)
                processed_indices.update(row_group)

    # Convert back to DataFrame
    result_df = pd.DataFrame(combined_rows)

    # Reorder columns to match original
    if not result_df.empty:
        result_df = result_df.reindex(columns=df.columns, fill_value=np.nan)

    return result_df

def find_lift_value_groups_with_tolerance(cancer_group: pd.DataFrame, lift_tolerance: float) -> List[List]:
    """
    Group rows by similar lift values within the tolerance.
    """

    rows = cancer_group.index.tolist()
    lift_values = cancer_group['Lift Value'].values
    n = len(rows)

    if n <= 1:
        return [rows]

    # Use Union-Find to group rows with similar lift values
    parent = list(range(n))

    def find(x):
        if parent[x] != x:
            parent[x] = find(parent[x])
        return parent[x]

    def union(x, y):
        px, py = find(x), find(y)
        if px != py:
            parent[px] = py

    # Union rows with lift values within tolerance
    for i in range(n):
        for j in range(i + 1, n):
            if abs(lift_values[i] - lift_values[j]) <= lift_tolerance:
                union(i, j)

    # Group rows by their root parent
    groups = defaultdict(list)
    for i in range(n):
        root = find(i)
        groups[root].append(rows[i])

    return list(groups.values())

def find_combinable_groups_tolerance(group: pd.DataFrame, matching_cols: List[str]) -> List[List]:
    """
    Find groups of rows that can be combined based on matching criteria.
    Uses Union-Find algorithm to efficiently group compatible rows.
    """

    rows = group.index.tolist()
    n = len(rows)

    if n <= 1:
        return [[row] for row in rows]

    # Create compatibility matrix
    compatible = [[False] * n for _ in range(n)]

    # Check pairwise compatibility
    for i in range(n):
        for j in range(i + 1, n):
            row1_idx, row2_idx = rows[i], rows[j]
            if can_combine_rows_tolerance(group.loc[[row1_idx, row2_idx]], matching_cols):
                compatible[i][j] = compatible[j][i] = True

    # Use Union-Find to group compatible rows
    parent = list(range(n))

    def find(x):
        if parent[x] != x:
            parent[x] = find(parent[x])
        return parent[x]

    def union(x, y):
        px, py = find(x), find(y)
        if px != py:
            parent[px] = py

    # Union compatible rows
    for i in range(n):
        for j in range(i + 1, n):
            if compatible[i][j]:
                union(i, j)

    # Group rows by their root parent
    groups = defaultdict(list)
    for i in range(n):
        root = find(i)
        groups[root].append(rows[i])

    # Verify each group is fully compatible
    final_groups = []
    for group_indices in groups.values():
        if len(group_indices) == 1:
            final_groups.append(group_indices)
        else:
            # Verify full compatibility within the group
            if verify_group_compatibility_tolerance(group.loc[group_indices], matching_cols):
                final_groups.append(group_indices)
            else:
                # If not fully compatible, split into individual rows
                final_groups.extend([[idx] for idx in group_indices])

    return final_groups

def can_combine_rows_tolerance(rows: pd.DataFrame, matching_cols: List[str]) -> bool:
    """
    Check if two or more rows can be combined based on matching criteria.
    """

    matching_count = 0

    for col in matching_cols:
        col_values = rows[col].dropna()

        if len(col_values) == 0:
            continue
        elif len(col_values) == 1:
            # Only one non-null value, no conflict
            continue
        elif len(set(col_values)) == 1:
            # All non-null values are the same
            matching_count += 1
        else:
            # Contradiction: different non-null values
            return False

    return matching_count >= 3

def verify_group_compatibility_tolerance(group: pd.DataFrame, matching_cols: List[str]) -> bool:
    """
    Verify that all rows in a group are mutually compatible.
    """

    matching_count = 0

    for col in matching_cols:
        col_values = group[col].dropna()

        if len(col_values) <= 1:
            continue
        elif len(set(col_values)) == 1:
            matching_count += 1
        else:
            return False

    return matching_count >= 3

def combine_row_group_with_lift_avg(group: pd.DataFrame, id_col: str = None) -> Dict[str, Any]:
    """
    Combine a group of compatible rows into a single row.
    Averages lift values when combining rows with different lift values.
    """

    combined = {}

    # Columns that should not be concatenated
    single_value_cols = {'Cancer Type'}

    for col in group.columns:
        non_null_values = group[col].dropna()

        if len(non_null_values) == 0:
            combined[col] = np.nan
        elif col == id_col and id_col is not None:
            # For ID column, combine all IDs (handle both string and numeric IDs)
            if isinstance(non_null_values.iloc[0], str) and ',' in str(non_null_values.iloc[0]):
                # Already combined IDs, merge them
                all_ids = []
                for val in non_null_values:
                    all_ids.extend(str(val).split(','))
                combined[col] = ','.join(sorted(set(all_ids)))
            else:
                combined[col] = ','.join(map(str, sorted(set(non_null_values.tolist()))))
        elif col in single_value_cols:
            # For Cancer Type, take single value (should be same for all)
            combined[col] = non_null_values.iloc[0]
        elif col == 'Lift Value':
            # For Lift Value, take the average of all values
            combined[col] = round(non_null_values.mean(), 2)
        else:
            # For other columns, take the first non-null value
            combined[col] = non_null_values.iloc[0]

    return combined

# Example usage combining both functions:
def full_combine_pipeline(df: pd.DataFrame, lift_tolerance: float = 1.0) -> pd.DataFrame:
    """
    Complete pipeline: first combine exact matches, then combine with lift tolerance.
    """

    # First pass: combine exact matches (assuming you have the original function)
    # combined_df = combine_cancer_data_rows(df)  # Your original function

    # Second pass: combine with lift tolerance
    final_df = combine_with_lift_tolerance(df, lift_tolerance)

    return final_df.sort_values(by="Lift Value", ascending=False)


df_after_first_combine = pd.DataFrame(combined_df)
final_result = combine_with_lift_tolerance(df_after_first_combine, lift_tolerance=1.0)
final_result.sort_values(by="Lift Value", ascending=False)

KeyboardInterrupt: 

In [61]:
merged_df.sort_values(by=["Lift Value", "Cancer Type"], ascending=[False, True])
# df_transformed.sort_values(by=["Lift Value", "Cancer Type"], ascending=[False, True])

,Cancer Type,Lift Value,Sex,Smoke Status,Chromosome,Hugo_Symbol,SNP_event,Diagnosis Age,VAR_TYPE_SX,Codons,Protein_position,Position,Consequence,Exon_Number,TMB (nonsynonymous)
0,Pancreatic Cancer,27.03,Female,Unknown,12,KRAS,C>G,71-80,Substitution/Indel,Ggt/Cgt,12.0,25398285.0-25398285.0,missense_variant,02/05,0.133333
16,Pancreatic Cancer,27.03,Female,Unknown,12,KRAS,C>G,61-70,Substitution/Indel,Ggt/Cgt,12.0,25398285.0-25398285.0,missense_variant,02/05,0.100000
44,Pancreatic Cancer,27.03,Female,Unknown,12,KRAS,C>G,51-60,Substitution/Indel,Ggt/Cgt,12.0,25398285.0-25398285.0,missense_variant,02/05,0.100000
140,Pancreatic Cancer,26.69,Female,Unknown,12,KRAS,C>G,61-70,Substitution/Indel,Ggt/Cgt,12.0,25398285.0-25398285.0,missense_variant,02/05,0.100000
0,Breast Carcinoma,26.22,Female,Unknown,3,PIK3CA,A>G,41-50,Substitution/Indel,cAt/cGt,1047.0,178952085.0-178952085.0,missense_variant,21/21,0.133333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1743,Breast Carcinoma,6.05,None,Unknown,16,TSC2,G>A,None,Substitution/Indel,None,NaN,None,None,None,NaN
114,Small Cell Lung Cancer,6.05,Female,Nonsmoker,None,None,C>G,51-60,Substitution/Indel,None,NaN,None,None,None,NaN
5861,Pancreatic Cancer,6.03,None,Unknown,12,KRAS,None,None,None,None,NaN,25398284.0-25398284.0,None,None,0.300000
56,Esophageal Carcinoma,6.02,None,Unknown,22,EP300,None,51-60,None,None,NaN,None,missense_variant,None,NaN


In [220]:
def row_to_sentence(row):
    return " AND ".join(
        f"{col} value is {val}"
        for col, val in row.items()
        if pd.notnull(val) and col not in {"Cancer Type", "Lift Value", "hypo_factors"}
    )

In [221]:
merged_df['hypo_factors'] = merged_df.apply(row_to_sentence, axis=1)
merged_df.rename(columns={'Lift Value': 'support'}, inplace=True)

In [222]:
merged_df

,Cancer Type,support,Sex,Smoke Status,Chromosome,Hugo_Symbol,SNP_event,Consequence,Exon_Number,Diagnosis Age,Position,Protein_position,Codons,VAR_TYPE_SX,TMB (nonsynonymous),hypo_factors
0,Breast Carcinoma,26.22,Female,Unknown,3,PIK3CA,A>G,missense_variant,21/21,41-50,178952085.0-178952085.0,1047.0,cAt/cGt,Substitution/Indel,0.100000,Sex value is Female AND Smoke Status value is ...
11,Breast Carcinoma,12.17,Female,Unknown,7,KMT2C,NaN,frameshift_variant,NaN,NaN,NaN,NaN,NaN,Truncation,NaN,Sex value is Female AND Smoke Status value is ...
12,Breast Carcinoma,11.65,Female,Unknown,17,TP53,C>T,missense_variant,06/11,41-50,NaN,NaN,NaN,Substitution/Indel,NaN,Sex value is Female AND Smoke Status value is ...
13,Breast Carcinoma,11.48,Female,Unknown,3,PIK3CA,G>A,missense_variant,10/21,NaN,178936082.0-178936082.0,542.0,Gaa/Aaa,Substitution/Indel,NaN,Sex value is Female AND Smoke Status value is ...
18,Breast Carcinoma,10.48,Female,Unknown,10,PTEN,None,None,None,None,None,NaN,None,Truncation,NaN,Sex value is Female AND Smoke Status value is ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,Small Cell Lung Cancer,6.12,Male,Unknown,13,RB1,None,frameshift_variant,None,None,None,NaN,None,Truncation,NaN,Sex value is Male AND Smoke Status value is Un...
0,Soft Tissue Sarcoma,14.73,Male,Unknown,4,KIT,NaN,missense_variant,NaN,41-50,NaN,NaN,NaN,Substitution/Indel,0.066667,Sex value is Male AND Smoke Status value is Un...
1,Soft Tissue Sarcoma,9.40,Female,Unknown,3,CTNNB1,A>G,missense_variant,03/15,41-50,41266124.0-41266124.0,41.0,Acc/Gcc,Substitution/Indel,0.100000,Sex value is Female AND Smoke Status value is ...
9,Soft Tissue Sarcoma,6.33,Female,Unknown,5,TERT,G>A,upstream_gene_variant,NaN,NaN,NaN,NaN,NaN,Substitution/Indel,NaN,Sex value is Female AND Smoke Status value is ...


In [67]:
merged_df.to_csv("models_hypotheses/LIFT_hypotheses_as_sentences.csv")

In [75]:
tal = pd.read_csv("models_hypotheses/LIFT_hypotheses_as_sentences.csv")
tal.sort_values(by="support", ascending=False)

,Unnamed: 0,Cancer Type,support,Sex,Smoke Status,Chromosome,Hugo_Symbol,SNP_event,Consequence,Exon_Number,Diagnosis Age,Position,Protein_position,Codons,VAR_TYPE_SX,TMB (nonsynonymous),hypo_factors
50,50,Pancreatic Cancer,27.03,Female,Unknown,12,KRAS,C>G,missense_variant,02/05,51-60,25398285.0-25398285.0,12.0,Ggt/Cgt,Substitution/Indel,0.066667,Sex value is Female AND Smoke Status value is ...
0,0,Breast Carcinoma,26.22,Female,Unknown,3,PIK3CA,A>G,missense_variant,21/21,41-50,178952085.0-178952085.0,1047.0,cAt/cGt,Substitution/Indel,0.100000,Sex value is Female AND Smoke Status value is ...
69,69,Small Cell Lung Cancer,20.05,Male,Smoker,13,RB1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Truncation,NaN,Sex value is Male AND Smoke Status value is Sm...
38,38,Intrahepatic Cholangiocarcinoma,16.57,Female,Unknown,2,IDH1,NaN,missense_variant,04/10,61-70,NaN,132.0,NaN,Substitution/Indel,0.100000,Sex value is Female AND Smoke Status value is ...
51,51,Pancreatic Cancer,16.25,Male,Unknown,12,KRAS,C>A,missense_variant,02/05,51-60,25398284.0-25398284.0,12.0,gGt/gTt,Substitution/Indel,NaN,Sex value is Male AND Smoke Status value is Un...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,27,Extrahepatic Cholangiocarcinoma,6.09,Male,Unknown,12,KRAS,C>A,missense_variant,NaN,61-70,NaN,NaN,NaN,Substitution/Indel,NaN,Sex value is Male AND Smoke Status value is Un...
67,67,Pancreatic Cancer,6.09,Female,Unknown,9,CDKN2A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Substitution/Indel,NaN,Sex value is Female AND Smoke Status value is ...
23,23,Esophageal Carcinoma,6.08,Male,Unknown,9,NOTCH1,C>T,missense_variant,NaN,NaN,NaN,NaN,NaN,Substitution/Indel,NaN,Sex value is Male AND Smoke Status value is Un...
28,28,Extrahepatic Cholangiocarcinoma,6.06,Female,Unknown,NaN,NaN,NaN,missense_variant,NaN,61-70,NaN,NaN,NaN,Substitution/Indel,0.133333,Sex value is Female AND Smoke Status value is ...


In [77]:
t = data_for_lift[data_for_lift["Smoke Status"] == "Unknown"]
t = t[t["Chromosome"] == "12"]
t = t[t["Hugo_Symbol"] == "KRAS"]
t = t[t["VAR_TYPE_SX"] == 'Substitution/Indel']
t = t[t["Protein_position"] == 12]
t = t[t["Exon_Number"] == "02/05"]
t = t[t["Codons"] == "Ggt/Cgt"]
t = t[t["SNP_event"] == "C>G"]
t = t[t["Consequence"] == "missense_variant"]
t = t[t["Diagnosis Age"] == "51-60"]
t = t[t["Position"] == "25398285.0-25398285.0"]
t

,Cancer Type,Cancer Type Detailed,Tumor Stage,Sample Type,Sex,Diagnosis Age,Smoke Status,TMB (nonsynonymous),Hugo_Symbol,Chromosome,...,Extrahepatic Cholangiocarcinoma,Gallbladder Carcinoma,Gastric Cancer,Intrahepatic Cholangiocarcinoma,Liver Hepatocellular Carcinoma,Non Small Cell Lung Cancer,Pancreatic Cancer,Small Cell Lung Cancer,Soft Tissue Sarcoma,Position
PATIENT_ID,,,,,,,,,,,,,,,,,,,,,
Patient0295,Non Small Cell Lung Cancer,Large Cell Lung Carcinoma,IV,Metastasis,Male,51-60,Unknown,1.866667,KRAS,12,...,0,0,0,0,0,1,0,0,0,25398285.0-25398285.0
Patient0378,Intrahepatic Cholangiocarcinoma,Intrahepatic Cholangiocarcinoma,Unknown,Primary,Male,51-60,Unknown,0.233333,KRAS,12,...,0,0,0,1,0,0,0,0,0,25398285.0-25398285.0
Patient0653,Pancreatic Cancer,Pancreatic Adenocarcinoma,IV,Metastasis,Female,51-60,Unknown,0.166667,KRAS,12,...,0,0,0,0,0,0,1,0,0,25398285.0-25398285.0
Patient1047,Pancreatic Cancer,Pancreatic Cancer Other,II,Primary,Male,51-60,Unknown,0.100000,KRAS,12,...,0,0,0,0,0,0,1,0,0,25398285.0-25398285.0
Patient1200,Pancreatic Cancer,Pancreatic Adenocarcinoma,IV,Metastasis,Female,51-60,Unknown,0.300000,KRAS,12,...,0,0,0,0,0,0,1,0,0,25398285.0-25398285.0
Patient1393,Intrahepatic Cholangiocarcinoma,Intrahepatic Cholangiocarcinoma,III,Primary,Female,51-60,Unknown,0.166667,KRAS,12,...,0,0,0,1,0,0,0,0,0,25398285.0-25398285.0
Patient1416,Pancreatic Cancer,Pancreatic Adenocarcinoma,IV,Metastasis,Male,51-60,Unknown,0.133333,KRAS,12,...,0,0,0,0,0,0,1,0,0,25398285.0-25398285.0
Patient1457,Pancreatic Cancer,Pancreatic Adenocarcinoma,Unknown,Primary,Female,51-60,Unknown,0.133333,KRAS,12,...,0,0,0,0,0,0,1,0,0,25398285.0-25398285.0
Patient1658,Pancreatic Cancer,Pancreatic Adenocarcinoma,II,Primary,Male,51-60,Unknown,0.166667,KRAS,12,...,0,0,0,0,0,0,1,0,0,25398285.0-25398285.0


In [66]:
merged_df.to_csv("data_for_lift_merged.csv")

In [134]:
lift_merged = pd.read_csv("lifts_merged.csv", index_col=0)

In [136]:
lift_merged.sort_values(by="Lift Value", ascending=False)

np.int64(0)

In [71]:
tt = merged_df[merged_df["Cancer Type"] == "Intrahepatic Cholangiocarcinoma"]
# tt = tt[tt["Hugo_Symbolmbol"] == "KRAS"]
# tt = tt[tt["Smoke Status"] == "Unknown"]
# tt = tt[tt["Chromosome"] == "12"]
# tt = tt[tt["VAR_TYPE_SX"] == 'Substitution/Indel']
tt = tt[tt["Protein_position"] == 249]
tt

,Cancer Type,Lift Value,Sex,Smoke Status,Chromosome,Hugo_Symbol,SNP_event,Consequence,Exon_Number,Diagnosis Age,Position,Protein_position,Codons,VAR_TYPE_SX,TMB (nonsynonymous)
71,Intrahepatic Cholangiocarcinoma,3.06,Male,Unknown,17,TP53,C>A,missense_variant,07/11,41-50,7577534.0-7577534.0,249.0,agG/agT,Substitution/Indel,NaN


In [ ]:
('Female', 'Unknown', '12', 'KRAS', 'Substitution/Indel')

In [49]:
l = lift_merged[lift_merged['Feature'].astype(str).str.contains('249', na=False)]
l = l[l.index == "Intrahepatic Cholangiocarcinoma"]
l

,Feature Combination,Feature,Lift Value
Cancer Type,,,
Intrahepatic Cholangiocarcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Male', 'Unknown', '17', 'TP53', 'C>A', 'miss...",3.06
Intrahepatic Cholangiocarcinoma,"('Smoke Status', 'Chromosome', 'Hugo_Symbol', ...","('Unknown', '17', 'TP53', 'missense_variant', ...",2.88
Intrahepatic Cholangiocarcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Male', 'Unknown', '17', 'TP53', 'C>A', 'miss...",3.06
Intrahepatic Cholangiocarcinoma,"('Smoke Status', 'Chromosome', 'Hugo_Symbol', ...","('Unknown', '17', 'TP53', 'C>A', 'missense_var...",3.16


In [22]:
df = lift_merged.copy()
df.reset_index(inplace=True)

In [39]:
df[df["Lift Value"] == 27.03]

,Cancer Type,Feature Combination,Feature,Lift Value
12652,Pancreatic Cancer,"(Smoke Status, Chromosome, Hugo_Symbol, SNP_ev...","(Unknown, 12, KRAS, C>G, missense_variant, 02/...",27.03


In [77]:
df_new = pd.read_csv("lifts_second_round.csv", index_col=0)

In [78]:
df_new

,Cancer Type,Feature Combination,Feature,Lift Value
0,Breast Carcinoma,"('Sex', 'Chromosome', 'Hugo_Symbol', 'SNP_even...","('Male', '12', 'KRAS', 'C>T', 'missense_varian...",0.00
1,Breast Carcinoma,"('Sex', 'Chromosome', 'Hugo_Symbol', 'SNP_even...","('Female', '7', 'EGFR', 'T>G', 'missense_varia...",0.00
2,Breast Carcinoma,"('Sex', 'Chromosome', 'Hugo_Symbol', 'SNP_even...","('Female', '12', 'KRAS', 'C>T', 'missense_vari...",0.00
3,Breast Carcinoma,"('Sex', 'Chromosome', 'Hugo_Symbol', 'SNP_even...","('Male', '12', 'KRAS', 'C>A', 'missense_varian...",0.00
4,Breast Carcinoma,"('Sex', 'Chromosome', 'Hugo_Symbol', 'SNP_even...","('Male', '17', 'TP53', 'C>T', 'missense_varian...",0.00
...,...,...,...,...
8331979,Soft Tissue Sarcoma,"('Sex', 'Chromosome', 'Hugo_Symbol', 'Conseque...","('Female', '1', 'TIE1', 'missense_variant', 'S...",0.00
8331980,Soft Tissue Sarcoma,"('Sex', 'Chromosome', 'Hugo_Symbol', 'Conseque...","('Male', '5', 'SDHA', 'missense_variant', 'Sub...",0.83
8331981,Soft Tissue Sarcoma,"('Sex', 'Chromosome', 'Hugo_Symbol', 'Conseque...","('Male', 'X', 'RBM10', 'stop_gained', 'Truncat...",0.00
8331982,Soft Tissue Sarcoma,"('Sex', 'Chromosome', 'Hugo_Symbol', 'Conseque...","('Male', 'X', 'RBM10', 'frameshift_variant', '...",0.84


In [8]:
import pandas as pd
from itertools import combinations

columns = [
    'Sex', 'Smoke Status', 'Chromosome', 'Hugo_Symbol', 'SNP_event',
    "Consequence", 'Exon_Number', "Diagnosis Age", "TMB (nonsynonymous)",
    "Position", "Protein_position", "Codons", "VAR_TYPE_SX"
]

result_frames = []

for col_set in combinations(columns, 4):
    # Group by the 4 columns and count unique PATIENT_IDs
    group = data_for_lift.groupby(list(col_set))['PATIENT_ID'].nunique().reset_index()
    # Filter for combinations with at least 50 unique patients
    filtered = group[group['PATIENT_ID'] >= 50]
    if not filtered.empty:
        filtered['Columns'] = ','.join(col_set)
        result_frames.append(filtered)

# Concatenate all results
combinations_df = pd.concat(result_frames, ignore_index=True)
combinations_df.head()

,Sex,Smoke Status,Chromosome,Hugo_Symbol,PATIENT_ID,Columns,SNP_event,Consequence,Exon_Number,Diagnosis Age,TMB (nonsynonymous),Position,Protein_position,Codons,VAR_TYPE_SX
0,Female,Nonsmoker,17,TP53,305,"Sex,Smoke Status,Chromosome,Hugo_Symbol",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Female,Nonsmoker,2,LRP1B,66,"Sex,Smoke Status,Chromosome,Hugo_Symbol",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Female,Nonsmoker,7,EGFR,497,"Sex,Smoke Status,Chromosome,Hugo_Symbol",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Female,Nonsmoker,X,RBM10,93,"Sex,Smoke Status,Chromosome,Hugo_Symbol",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Female,Unknown,1,ARID1A,267,"Sex,Smoke Status,Chromosome,Hugo_Symbol",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
# combinations_df.rename(columns={'PATIENT_ID': 'patient_ids_count'}, inplace=True)
combinations_df

,Sex,Smoke Status,Chromosome,Hugo_Symbol,patient_ids_count,Columns,SNP_event,Consequence,Exon_Number,Diagnosis Age,TMB (nonsynonymous),Position,Protein_position,Codons,VAR_TYPE_SX,Cancer Type
0,Female,Nonsmoker,17,TP53,305,"Sex,Smoke Status,Chromosome,Hugo_Symbol",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Colorectal Carcinoma
1,Female,Nonsmoker,17,TP53,305,"Sex,Smoke Status,Chromosome,Hugo_Symbol",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Liver Hepatocellular Carcinoma
2,Female,Nonsmoker,17,TP53,305,"Sex,Smoke Status,Chromosome,Hugo_Symbol",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Soft Tissue Sarcoma
3,Female,Nonsmoker,17,TP53,305,"Sex,Smoke Status,Chromosome,Hugo_Symbol",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gastric Cancer
4,Female,Nonsmoker,17,TP53,305,"Sex,Smoke Status,Chromosome,Hugo_Symbol",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pancreatic Cancer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541867,NaN,NaN,NaN,NaN,251,"Position,Protein_position,Codons,VAR_TYPE_SX",NaN,NaN,NaN,NaN,NaN,7578406.0-7578406.0,175.0,cGc/cAc,Substitution/Indel,Esophageal Carcinoma
541868,NaN,NaN,NaN,NaN,251,"Position,Protein_position,Codons,VAR_TYPE_SX",NaN,NaN,NaN,NaN,NaN,7578406.0-7578406.0,175.0,cGc/cAc,Substitution/Indel,Small Cell Lung Cancer
541869,NaN,NaN,NaN,NaN,251,"Position,Protein_position,Codons,VAR_TYPE_SX",NaN,NaN,NaN,NaN,NaN,7578406.0-7578406.0,175.0,cGc/cAc,Substitution/Indel,Intrahepatic Cholangiocarcinoma
541870,NaN,NaN,NaN,NaN,251,"Position,Protein_position,Codons,VAR_TYPE_SX",NaN,NaN,NaN,NaN,NaN,7578406.0-7578406.0,175.0,cGc/cAc,Substitution/Indel,Non Small Cell Lung Cancer


In [13]:
cancer_types = pd.DataFrame({'Cancer Type': data_for_lift['Cancer Type'].unique()})
combinations_df = combinations_df.merge(cancer_types, how='cross')

In [19]:
df = find_matching_patients(combinations_df, data_for_lift)


Matching patients: 100%|██████████| 541872/541872 [33:39<00:00, 268.27it/s]  


In [30]:
from tqdm import tqdm

def find_matching_patients(df_transformed, data_for_lift):
    df_transformed = df_transformed.copy()
    data_for_lift = data_for_lift.copy()
    cancer_type_groups = data_for_lift.groupby('Cancer Type')
    matched_ids = []
    matched_counts = []
    total_matched_feature_counts = []

    for idx, row in tqdm(df_transformed.iterrows(), total=len(df_transformed), desc="Matching patients"):
        cancer_type = row['Cancer Type']
        filter_columns = row.drop(labels=['Cancer Type']).dropna()

        # Find matched patient IDs for this cancer type
        if cancer_type not in cancer_type_groups.groups:
            matched = []
        else:
            subset = cancer_type_groups.get_group(cancer_type)
            if filter_columns.empty:
                matched = subset['PATIENT_ID'].dropna().unique()
            else:
                mask = pd.Series(True, index=subset.index)
                for col, val in filter_columns.items():
                    if col in subset.columns:
                        mask &= subset[col] == val
                matched = subset.loc[mask, 'PATIENT_ID'].dropna().unique()
        matched_ids.append(','.join(map(str, matched)))
        matched_counts.append(len(matched))

        # Find total matched patient IDs with these features across all cancer types
        if filter_columns.empty:
            total_matched = data_for_lift['PATIENT_ID'].dropna().unique()
        else:
            mask = pd.Series(True, index=data_for_lift.index)
            for col, val in filter_columns.items():
                if col in data_for_lift.columns:
                    mask &= data_for_lift[col] == val
            total_matched = data_for_lift.loc[mask, 'PATIENT_ID'].dropna().unique()
        total_matched_feature_counts.append(len(total_matched))

    df_transformed['Matched PATIENT_IDs'] = matched_ids
    df_transformed['Matched_Count'] = matched_counts
    df_transformed['Total_Matched_Feature_Count'] = total_matched_feature_counts
    return df_transformed

In [36]:
df = find_matching_patients(combinations_df, data_for_lift)

Matching patients: 100%|██████████| 541872/541872 [1:33:43<00:00, 96.36it/s] 


In [50]:
df

,Sex,Smoke Status,Chromosome,Hugo_Symbol,patient_ids_count,Columns,SNP_event,Consequence,Exon_Number,Diagnosis Age,TMB (nonsynonymous),Position,Protein_position,Codons,VAR_TYPE_SX,Cancer Type,Matched PATIENT_IDs,Matched_Count,Total_Matched_Feature_Count
0,Female,Nonsmoker,17,TP53,305,"Sex,Smoke Status,Chromosome,Hugo_Symbol",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Colorectal Carcinoma,,0,305
1,Female,Nonsmoker,17,TP53,305,"Sex,Smoke Status,Chromosome,Hugo_Symbol",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Liver Hepatocellular Carcinoma,,0,305
2,Female,Nonsmoker,17,TP53,305,"Sex,Smoke Status,Chromosome,Hugo_Symbol",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Soft Tissue Sarcoma,,0,305
3,Female,Nonsmoker,17,TP53,305,"Sex,Smoke Status,Chromosome,Hugo_Symbol",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gastric Cancer,,0,305
4,Female,Nonsmoker,17,TP53,305,"Sex,Smoke Status,Chromosome,Hugo_Symbol",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pancreatic Cancer,,0,305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541867,NaN,NaN,NaN,NaN,251,"Position,Protein_position,Codons,VAR_TYPE_SX",NaN,NaN,NaN,NaN,NaN,7578406.0-7578406.0,175.0,cGc/cAc,Substitution/Indel,Esophageal Carcinoma,"Patient0367,Patient0533,Patient1714,Patient174...",21,251
541868,NaN,NaN,NaN,NaN,251,"Position,Protein_position,Codons,VAR_TYPE_SX",NaN,NaN,NaN,NaN,NaN,7578406.0-7578406.0,175.0,cGc/cAc,Substitution/Indel,Small Cell Lung Cancer,"Patient7290,Patient8250,Patient8324,Patient832...",7,251
541869,NaN,NaN,NaN,NaN,251,"Position,Protein_position,Codons,VAR_TYPE_SX",NaN,NaN,NaN,NaN,NaN,7578406.0-7578406.0,175.0,cGc/cAc,Substitution/Indel,Intrahepatic Cholangiocarcinoma,"Patient0213,Patient0857,Patient2178,Patient228...",9,251
541870,NaN,NaN,NaN,NaN,251,"Position,Protein_position,Codons,VAR_TYPE_SX",NaN,NaN,NaN,NaN,NaN,7578406.0-7578406.0,175.0,cGc/cAc,Substitution/Indel,Non Small Cell Lung Cancer,"Patient0140,Patient0729,Patient0916,Patient095...",24,251


In [53]:
df = df[df["Matched_Count"] > 0]
# Calculate unique patient counts per cancer type
cancer_type_patient_counts = data_for_lift.groupby('Cancer Type')['PATIENT_ID'].nunique()
df['Total_Unique_Patients_Cancer_Type'] = df['Cancer Type'].map(cancer_type_patient_counts)

total_patients = data_for_lift['PATIENT_ID'].nunique()

P_A = df['Total_Unique_Patients_Cancer_Type'] / total_patients
P_B = df['Total_Matched_Feature_Count'] / total_patients
P_A_B = df['Matched_Count'] / total_patients

df["lift_try"] = P_A_B / (P_A * P_B)
df

,Sex,Smoke Status,Chromosome,Hugo_Symbol,patient_ids_count,Columns,SNP_event,Consequence,Exon_Number,Diagnosis Age,...,Position,Protein_position,Codons,VAR_TYPE_SX,Cancer Type,Matched PATIENT_IDs,Matched_Count,Total_Matched_Feature_Count,Total_Unique_Patients_Cancer_Type,lift_try
8,Female,Nonsmoker,17,TP53,305,"Sex,Smoke Status,Chromosome,Hugo_Symbol",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Small Cell Lung Cancer,"Patient1998,Patient2372,Patient3725,Patient452...",26,305,215,3.320625
10,Female,Nonsmoker,17,TP53,305,"Sex,Smoke Status,Chromosome,Hugo_Symbol",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Non Small Cell Lung Cancer,"Patient0112,Patient0117,Patient0124,Patient014...",279,305,2002,3.826706
20,Female,Nonsmoker,2,LRP1B,66,"Sex,Smoke Status,Chromosome,Hugo_Symbol",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Small Cell Lung Cancer,"Patient7120,Patient8233,Patient8259,Patient826...",13,66,215,7.672657
22,Female,Nonsmoker,2,LRP1B,66,"Sex,Smoke Status,Chromosome,Hugo_Symbol",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Non Small Cell Lung Cancer,"Patient0084,Patient0210,Patient0680,Patient123...",53,66,2002,3.359330
32,Female,Nonsmoker,7,EGFR,497,"Sex,Smoke Status,Chromosome,Hugo_Symbol",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Small Cell Lung Cancer,Patient8016,1,497,215,0.078377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541867,NaN,NaN,NaN,NaN,251,"Position,Protein_position,Codons,VAR_TYPE_SX",NaN,NaN,NaN,NaN,...,7578406.0-7578406.0,175.0,cGc/cAc,Substitution/Indel,Esophageal Carcinoma,"Patient0367,Patient0533,Patient1714,Patient174...",21,251,601,1.165886
541868,NaN,NaN,NaN,NaN,251,"Position,Protein_position,Codons,VAR_TYPE_SX",NaN,NaN,NaN,NaN,...,7578406.0-7578406.0,175.0,cGc/cAc,Substitution/Indel,Small Cell Lung Cancer,"Patient7290,Patient8250,Patient8324,Patient832...",7,251,215,1.086352
541869,NaN,NaN,NaN,NaN,251,"Position,Protein_position,Codons,VAR_TYPE_SX",NaN,NaN,NaN,NaN,...,7578406.0-7578406.0,175.0,cGc/cAc,Substitution/Indel,Intrahepatic Cholangiocarcinoma,"Patient0213,Patient0857,Patient2178,Patient228...",9,251,545,0.551007
541870,NaN,NaN,NaN,NaN,251,"Position,Protein_position,Codons,VAR_TYPE_SX",NaN,NaN,NaN,NaN,...,7578406.0-7578406.0,175.0,cGc/cAc,Substitution/Indel,Non Small Cell Lung Cancer,"Patient0140,Patient0729,Patient0916,Patient095...",24,251,2002,0.399998


In [55]:
df = df.sort_values(by="lift_try", ascending=False)
unique_ids = df['Matched PATIENT_IDs'].drop_duplicates().reset_index(drop=True)
id_to_rank = {pid: i+1 for i, pid in unique_ids.items()}
df['patient_rank'] = df['Matched PATIENT_IDs'].map(id_to_rank)
df

,Sex,Smoke Status,Chromosome,Hugo_Symbol,patient_ids_count,Columns,SNP_event,Consequence,Exon_Number,Diagnosis Age,...,Protein_position,Codons,VAR_TYPE_SX,Cancer Type,Matched PATIENT_IDs,Matched_Count,Total_Matched_Feature_Count,Total_Unique_Patients_Cancer_Type,lift_try,patient_rank
35156,Male,Smoker,NaN,RB1,71,"Sex,Smoke Status,Hugo_Symbol,VAR_TYPE_SX",NaN,NaN,NaN,NaN,...,NaN,NaN,Truncation,Small Cell Lung Cancer,"Patient0238,Patient0492,Patient0632,Patient070...",47,71,215,25.786112,1
236984,NaN,Smoker,13,RB1,74,"Smoke Status,Chromosome,Hugo_Symbol,VAR_TYPE_SX",NaN,NaN,NaN,NaN,...,NaN,NaN,Truncation,Small Cell Lung Cancer,"Patient0238,Patient0492,Patient0632,Patient070...",48,74,215,25.267128,2
22280,Male,Smoker,13,NaN,85,"Sex,Smoke Status,Chromosome,VAR_TYPE_SX",NaN,NaN,NaN,NaN,...,NaN,NaN,Truncation,Small Cell Lung Cancer,"Patient0238,Patient0492,Patient0632,Patient070...",49,85,215,22.455540,3
1628,Male,Smoker,13,RB1,120,"Sex,Smoke Status,Chromosome,Hugo_Symbol",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Small Cell Lung Cancer,"Patient0125,Patient0238,Patient0492,Patient063...",68,120,215,22.073643,4
265640,NaN,Smoker,13,NaN,50,"Smoke Status,Chromosome,Consequence,VAR_TYPE_SX",NaN,stop_gained,NaN,NaN,...,NaN,NaN,Truncation,Small Cell Lung Cancer,"Patient0492,Patient0632,Patient0704,Patient101...",26,50,215,20.255814,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394843,NaN,NaN,12,KRAS,1429,"Chromosome,Hugo_Symbol,Exon_Number,VAR_TYPE_SX",NaN,NaN,02/05,NaN,...,NaN,NaN,Substitution/Indel,Esophageal Carcinoma,Patient3295,1,1429,601,0.009752,147490
431059,NaN,NaN,12,NaN,1429,"Chromosome,Consequence,Exon_Number,VAR_TYPE_SX",NaN,missense_variant,02/05,NaN,...,NaN,NaN,Substitution/Indel,Esophageal Carcinoma,Patient3295,1,1429,601,0.009752,147490
120797,Male,NaN,17,NaN,2768,"Sex,Chromosome,Consequence,VAR_TYPE_SX",NaN,missense_variant,NaN,NaN,...,NaN,NaN,Substitution/Indel,Breast Carcinoma,Patient2337,1,2768,313,0.009667,130337
175277,Male,NaN,NaN,NaN,2791,"Sex,SNP_event,Consequence,VAR_TYPE_SX",G>A,missense_variant,NaN,NaN,...,NaN,NaN,Substitution/Indel,Breast Carcinoma,Patient6849,1,2791,313,0.009587,130329


In [107]:
dff = df.copy()
dff = dff[dff["lift_try"] > 4]
dff["Matched_Count"] = dff["Matched_Count"].astype(int)
dff = dff[dff["Matched_Count"] >= 10]
dff

,Sex,Smoke Status,Chromosome,Hugo_Symbol,patient_ids_count,Columns,SNP_event,Consequence,Exon_Number,Diagnosis Age,...,Protein_position,Codons,VAR_TYPE_SX,Cancer Type,Matched PATIENT_IDs,Matched_Count,Total_Matched_Feature_Count,Total_Unique_Patients_Cancer_Type,lift_try,patient_rank
35156,Male,Smoker,NaN,RB1,71,"Sex,Smoke Status,Hugo_Symbol,VAR_TYPE_SX",NaN,NaN,NaN,NaN,...,NaN,NaN,Truncation,Small Cell Lung Cancer,"Patient0238,Patient0492,Patient0632,Patient070...",47,71,215,25.786112,1
236984,NaN,Smoker,13,RB1,74,"Smoke Status,Chromosome,Hugo_Symbol,VAR_TYPE_SX",NaN,NaN,NaN,NaN,...,NaN,NaN,Truncation,Small Cell Lung Cancer,"Patient0238,Patient0492,Patient0632,Patient070...",48,74,215,25.267128,2
22280,Male,Smoker,13,NaN,85,"Sex,Smoke Status,Chromosome,VAR_TYPE_SX",NaN,NaN,NaN,NaN,...,NaN,NaN,Truncation,Small Cell Lung Cancer,"Patient0238,Patient0492,Patient0632,Patient070...",49,85,215,22.455540,3
1628,Male,Smoker,13,RB1,120,"Sex,Smoke Status,Chromosome,Hugo_Symbol",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Small Cell Lung Cancer,"Patient0125,Patient0238,Patient0492,Patient063...",68,120,215,22.073643,4
265640,NaN,Smoker,13,NaN,50,"Smoke Status,Chromosome,Consequence,VAR_TYPE_SX",NaN,stop_gained,NaN,NaN,...,NaN,NaN,Truncation,Small Cell Lung Cancer,"Patient0492,Patient0632,Patient0704,Patient101...",26,50,215,20.255814,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248712,NaN,Unknown,5,NaN,55,"Smoke Status,Chromosome,SNP_event,Diagnosis Age",G>T,NaN,NaN,41-50,...,NaN,NaN,NaN,Colorectal Carcinoma,"Patient0462,Patient0482,Patient0507,Patient065...",32,55,1218,4.000597,4078
195096,Female,NaN,NaN,NaN,55,"Sex,Consequence,Exon_Number,VAR_TYPE_SX",NaN,missense_variant,04/05,NaN,...,NaN,NaN,Substitution/Indel,Colorectal Carcinoma,"Patient0003,Patient0173,Patient0200,Patient098...",32,55,1218,4.000597,3015
483216,NaN,NaN,NaN,KRAS,110,"Hugo_Symbol,Exon_Number,TMB (nonsynonymous),VA...",NaN,NaN,02/05,NaN,...,NaN,NaN,Substitution/Indel,Colorectal Carcinoma,"Patient0014,Patient0267,Patient0453,Patient051...",64,110,1218,4.000597,2641
447060,NaN,NaN,12,NaN,110,"Chromosome,Exon_Number,TMB (nonsynonymous),VAR...",NaN,NaN,02/05,NaN,...,NaN,NaN,Substitution/Indel,Colorectal Carcinoma,"Patient0014,Patient0267,Patient0453,Patient051...",64,110,1218,4.000597,2641


In [138]:
# Group by 'patient_rank' and aggregate unique values as comma-separated strings
agg_df = dff.groupby('patient_rank').agg(
    lambda x: ','.join(sorted(map(str, pd.unique(x.dropna()))))
).reset_index()

def get_max_from_str(val):
    if pd.isna(val):
        return None
    vals = [float(x) for x in str(val).split(',') if x.strip()]
    return max(vals) if vals else None

agg_df['max_lift'] = agg_df['lift_try'].apply(get_max_from_str)
agg_df.replace({"": None}, inplace=True)
agg_df["Matched_Count"] = agg_df["Matched_Count"].astype(int)
agg_df

,patient_rank,Sex,Smoke Status,Chromosome,Hugo_Symbol,patient_ids_count,Columns,SNP_event,Consequence,Exon_Number,...,Protein_position,Codons,VAR_TYPE_SX,Cancer Type,Matched PATIENT_IDs,Matched_Count,Total_Matched_Feature_Count,Total_Unique_Patients_Cancer_Type,lift_try,max_lift
0,1,Male,Smoker,None,RB1,71,"Sex,Smoke Status,Hugo_Symbol,VAR_TYPE_SX",None,None,None,...,None,None,Truncation,Small Cell Lung Cancer,"Patient0238,Patient0492,Patient0632,Patient070...",47,71,215,25.78611202096299,25.786112
1,2,None,Smoker,13,RB1,74,"Smoke Status,Chromosome,Hugo_Symbol,VAR_TYPE_SX",None,None,None,...,None,None,Truncation,Small Cell Lung Cancer,"Patient0238,Patient0492,Patient0632,Patient070...",48,74,215,25.267127592708988,25.267128
2,3,Male,Smoker,13,None,85,"Sex,Smoke Status,Chromosome,VAR_TYPE_SX",None,None,None,...,None,None,Truncation,Small Cell Lung Cancer,"Patient0238,Patient0492,Patient0632,Patient070...",49,85,215,22.455540355677154,22.455540
3,4,Male,Smoker,13,RB1,120,"Sex,Smoke Status,Chromosome,Hugo_Symbol",None,None,None,...,None,None,None,Small Cell Lung Cancer,"Patient0125,Patient0238,Patient0492,Patient063...",68,120,215,22.073643410852714,22.073643
4,5,None,Smoker,13,None,50,"Smoke Status,Chromosome,Consequence,VAR_TYPE_SX",None,stop_gained,None,...,None,None,Truncation,Small Cell Lung Cancer,"Patient0492,Patient0632,Patient0704,Patient101...",26,50,215,20.25581395348837,20.255814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3479,4074,Female,Nonsmoker,None,None,69,"Sex,Smoke Status,SNP_event,Consequence",C>T,stop_gained,None,...,None,None,None,Non Small Cell Lung Cancer,"Patient0391,Patient0419,Patient1443,Patient144...",66,69,2002,4.001433349259436,4.001433
3480,4075,Male,Smoker,None,None,185,"Sex,Smoke Status,SNP_event,VAR_TYPE_SX",T>G,None,None,...,None,None,Substitution/Indel,Small Cell Lung Cancer,"Patient0125,Patient0492,Patient1430,Patient227...",19,185,215,4.000628535512257,4.000629
3481,4076,None,Unknown,13,None,55,"Smoke Status,Chromosome,Consequence,Diagnosis Age",None,missense_variant,None,...,None,None,None,Colorectal Carcinoma,"Patient0153,Patient0173,Patient0200,Patient057...",32,55,1218,4.00059710404538,4.000597
3482,4077,None,Unknown,None,None,55,"Smoke Status,SNP_event,Consequence,Codons",G>A,missense_variant,None,...,None,tGc/tAc,None,Colorectal Carcinoma,"Patient0004,Patient0987,Patient0998,Patient105...",32,55,1218,4.00059710404538,4.000597


In [147]:
combine_cols = [
    'Sex', 'Smoke Status', 'Chromosome', 'Hugo_Symbol', 'SNP_event',
    "Consequence", 'Exon_Number', "Diagnosis Age", "TMB (nonsynonymous)",
    "Position", "Protein_position", "Codons", "VAR_TYPE_SX"
]

def patient_id_overlap(row1, row2):
    set1 = set(str(row1['Matched PATIENT_IDs']).split(','))
    set2 = set(str(row2['Matched PATIENT_IDs']).split(','))
    if not set1 or not set2:
        return False
    intersection = set1 & set2
    union = set1 | set2
    if len(union) == 0:
        return False
    return len(intersection) / len(union) >= 0.8

def merge_rows(df):
    result_rows = []
    used = set()
    df = df.sort_values( 'max_lift', ascending=False).reset_index(drop=True)

    for cancer_type, group in df.groupby('Cancer Type'):
        group = group.reset_index()  # preserve original index
        n = len(group)

        for i in range(n):
            idx_i = group.loc[i, 'index']
            if idx_i in used:
                continue

            base_row = group.loc[i]
            merge_candidates = [i]

            for j in range(n):
                if i == j:
                    continue

                idx_j = group.loc[j, 'index']
                if idx_j in used:
                    continue

                comp_row = group.loc[j]

                # Only check combine_cols for consistency
                if not patient_id_overlap(base_row, comp_row):
                    continue
                matches = 0
                consistent = True
                for col in combine_cols:
                    val1 = base_row[col]
                    val2 = comp_row[col]
                    if pd.notna(val1) and pd.notna(val2):
                        if val1 != val2:
                            consistent = False
                            break
                        matches += 1

                if consistent and matches >= 3 and patient_id_overlap(base_row, comp_row):
                    merge_candidates.append(j)

            merge_indices = [group.loc[k, 'index'] for k in merge_candidates]
            if len(merge_candidates) > 1:
                subset = group.loc[merge_candidates]
                merged_row = base_row.copy()
                # Aggregate all columns by concatenating unique, non-null values
                for col in subset.columns:
                    vals = subset[col].dropna().astype(str).unique()
                    if col in ["Columns", "Matched PATIENT_IDs"]:
                        unique_vals = set()
                        for v in vals:
                            unique_vals.update(map(str.strip, v.split(',')))
                        merged_row[col] = ','.join(sorted(unique_vals))
                    else:
                        merged_row[col] = ','.join(sorted(set(vals)))
                # Add patient_rank_agg and patient_rank_max
                patient_ranks = subset['patient_rank'].dropna().astype(int)
                merged_row['patient_rank_agg'] = ','.join(sorted(map(str, set(patient_ranks))))
                merged_row['patient_rank_min'] = patient_ranks.min() if not patient_ranks.empty else None
                result_rows.append(merged_row.drop(labels='index'))
                used.update(merge_indices)
            else:
                base_row = base_row.copy()
                # Single row: set both columns to its value
                if pd.notna(base_row['patient_rank']):
                    base_row['patient_rank_agg'] = str(int(base_row['patient_rank']))
                    base_row['patient_rank_min'] = int(base_row['patient_rank'])
                else:
                    base_row['patient_rank_agg'] = ''
                    base_row['patient_rank_max'] = None
                result_rows.append(base_row.drop(labels='index'))
                used.add(idx_i)

    return pd.DataFrame(result_rows)

merged_df = merge_rows(agg_df)

In [149]:
merged_df.replace("", np.nan, inplace=True)
merged_df['max_lift'] = merged_df['lift_try'].apply(get_max_from_str)
merged_df

,patient_rank,Sex,Smoke Status,Chromosome,Hugo_Symbol,patient_ids_count,Columns,SNP_event,Consequence,Exon_Number,...,VAR_TYPE_SX,Cancer Type,Matched PATIENT_IDs,Matched_Count,Total_Matched_Feature_Count,Total_Unique_Patients_Cancer_Type,lift_try,max_lift,patient_rank_agg,patient_rank_min
0,6,Female,Unknown,10,GATA3,69,"Sex,Smoke Status,Chromosome,Hugo_Symbol",None,None,None,...,None,Breast Carcinoma,"Patient0248,Patient0348,Patient0787,Patient082...",47,69,313,18.22591100615826,18.225911,6,6
1,"103,11,1382,15,151,16,213,22,222,225,25,27,347...",Female,Unknown,3,PIK3CA,"100,110,86,87,88,90,92,93,94,95,96,105,115,106...","Chromosome,Codons,Consequence,Exon_Number,Hugo...",A>G,missense_variant,21/21,...,Substitution/Indel,Breast Carcinoma,"Patient0041,Patient0074,Patient0114,Patient016...","55,56,57,58,59,60,63,64,65,66","100,110,86,87,88,90,92,93,94,95,96,105,115,106...",313,"10.033945686900958,8.725170162522573,9.0702333...",17.112155,"103,11,1382,15,151,16,213,22,222,225,25,27,347...",9
3,"13,23,24",Female,Unknown,3,PIK3CA,"129,143,83,136,151,85,138,81,86","Chromosome,Consequence,Diagnosis Age,Hugo_Symb...",NaN,missense_variant,NaN,...,Substitution/Indel,Breast Carcinoma,"Patient0041,Patient0074,Patient0129,Patient016...","44,45,46","129,143,83,136,151,85,138,81,86",313,"14.165570381507234,7.973996572371621,8.8534814...",15.195440,"13,23,24",13
6,"19,26,45",NaN,Unknown,3,PIK3CA,"50,51,51,62,65","Chromosome,Consequence,Diagnosis Age,Exon_Numb...",NaN,missense_variant,21/21,...,Substitution/Indel,Breast Carcinoma,"Patient0041,Patient0074,Patient0160,Patient032...","26,27","50,51,51,62,65",313,"11.1145244531826,11.652324023497886,13.6409196...",14.448882,"19,26,45",19
13,"42,65",NaN,Unknown,3,PIK3CA,"50,51,91,92,98,58,68","Chromosome,Codons,Consequence,Diagnosis Age,Hu...",A>G,missense_variant,NaN,...,Substitution/Indel,Breast Carcinoma,"Patient0041,Patient0074,Patient0160,Patient061...",22,"50,51,91,92,98,58,68",313,"10.149278395945798,8.656737455365532,11.542316...",11.773163,"42,65",42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39,3203,None,Unknown,None,None,57,"Smoke Status,Consequence,Exon_Number,VAR_TYPE_SX",None,missense_variant,11/21,...,Substitution/Indel,Soft Tissue Sarcoma,"Patient0095,Patient0365,Patient1324,Patient163...",13,57,454,4.207241672463096,4.207242,3203,3203
40,3448,Female,Unknown,None,None,84,"Sex,Smoke Status,TMB (nonsynonymous),VAR_TYPE_SX",None,None,None,...,Truncation,Soft Tissue Sarcoma,"Patient0096,Patient0542,Patient10099,Patient13...",19,84,454,4.172566603734006,4.172567,3448,3448
42,3734,None,None,2,None,54,"Chromosome,Consequence,TMB (nonsynonymous),VAR...",None,missense_variant,None,...,Substitution/Indel,Soft Tissue Sarcoma,"Patient0356,Patient1602,Patient2589,Patient390...",12,54,454,4.099363680861479,4.099364,3734,3734
43,3988,None,Unknown,None,None,55,"Smoke Status,Consequence,TMB (nonsynonymous),V...",None,"splice_region_variant,intron_variant",None,...,Substitution/Indel,Soft Tissue Sarcoma,"Patient0691,Patient2453,Patient2802,Patient282...",12,55,454,4.024829795754906,4.024830,3988,3988


In [142]:
dff[dff["patient_rank"].isin([1,2,3,4])]

,Sex,Smoke Status,Chromosome,Hugo_Symbol,patient_ids_count,Columns,SNP_event,Consequence,Exon_Number,Diagnosis Age,...,Protein_position,Codons,VAR_TYPE_SX,Cancer Type,Matched PATIENT_IDs,Matched_Count,Total_Matched_Feature_Count,Total_Unique_Patients_Cancer_Type,lift_try,patient_rank
35156,Male,Smoker,NaN,RB1,71,"Sex,Smoke Status,Hugo_Symbol,VAR_TYPE_SX",NaN,NaN,NaN,NaN,...,NaN,NaN,Truncation,Small Cell Lung Cancer,"Patient0238,Patient0492,Patient0632,Patient070...",47,71,215,25.786112,1
236984,NaN,Smoker,13,RB1,74,"Smoke Status,Chromosome,Hugo_Symbol,VAR_TYPE_SX",NaN,NaN,NaN,NaN,...,NaN,NaN,Truncation,Small Cell Lung Cancer,"Patient0238,Patient0492,Patient0632,Patient070...",48,74,215,25.267128,2
22280,Male,Smoker,13,NaN,85,"Sex,Smoke Status,Chromosome,VAR_TYPE_SX",NaN,NaN,NaN,NaN,...,NaN,NaN,Truncation,Small Cell Lung Cancer,"Patient0238,Patient0492,Patient0632,Patient070...",49,85,215,22.455540,3
1628,Male,Smoker,13,RB1,120,"Sex,Smoke Status,Chromosome,Hugo_Symbol",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Small Cell Lung Cancer,"Patient0125,Patient0238,Patient0492,Patient063...",68,120,215,22.073643,4


In [150]:
d = data_for_lift[data_for_lift["Smoke Status"] == "Smoker"]
d = d[d["Chromosome"] == "13"]
d = d[d["Hugo_Symbol"] == "RB1"]
# d = d[d["VAR_TYPE_SX"] == 'Truncation']
d["Diagnosis Age"].value_counts()

Diagnosis Age
61-70    59
51-60    39
41-50    17
71-80    15
31-40     3
21-30     2
80+       2
Name: count, dtype: int64